# ECE 208 Bonus Assignment

In [25]:
# all imports 
from typing import List, Dict, Tuple, Type, Any
# from google.colab import drive
# drive.mount('/gdrive')

In [2]:
# %ls /gdrive/'My Drive'/Waterloo/ECE208

### The NNF Format
First,  we define the textual format for Boolean formulas in negation normal form (NNF). 
It is assumed that you are familiar with the inductive definition of NNF formulas over connectives: $\neg,\; \land,\; \lor$ , and left/right parentheses.

* Every Boolean variable is represented as a 32-bit number, written in decimal (just as the int datatypein C++ or Java).  For example, the number ”1” corresponds to the Boolean variable $x_1$.  The number ”0” is reserved for end of line.
* Negated variables are represented by negative numbers, e.g.: $\neg x_1$ is represented as $\texttt{-1}$ , and $x_{199}$ is represented as $\texttt{-199}$.
* The Boolean connective $\lor$ (OR) is represented by + symbol.  The Boolean connective $\land$ (AND) is represented by the . symbol. 
    
You may use the left and right parentheses to demarcate sub-formulas.You may assume that the constants $\top$ and $\bot$ have been simplified away from all input formulas. If youreally need to represent these constants, you can use suitable formulas over variables (e.g., $\top$ is equivalent to
$(x)\lor(\neg x)$)

### The CNF Format
Here we define the textual format for Boolean formulas in conjunctive normal form (CNF). 
It is assumed that you are familiar with the inductive definition of CNF. (This text format is the official DIMACS formatused by all SAT solvers).
* Every Boolean variable is represented by a 32-bit number, exactly as in NNF. For example, the number ”1” corresponds to the Boolean variable $x_1$. The number ”0” is reserved for end of line.
* Negated variables are represented by negative numbers, e.g.,$\neg x_1$is represented as $\texttt{-1}$.
* Every clause is represented by a space separated list of numbers, ending in a 0 to denote end of line.That is, the Boolean connective OR in disjunctive clauses is represented by a white space.E.g., the clause 
$(x_{42} \vee \neg x_1 x_{100} \vee x_{22} 
\vee \neg x_{90})$ would be written as $\texttt{42 -1 100 22 -90 0}$.
* A formula in CNF is a list of clauses, one clause per line. (The AND is implicit.)

You may assume that the constants $\top$ and $\bot$ have been simplified away from all input formulas. If youreally need to represent these constants, you can use suitable formulas over variables (e.g.,$\bot$ is equivalent to $(x)\wedge (\neg x)$)

## Question 1: Tseitin Transformer (maximum 2 points)
Write a Tseitin transformer that takes as input a Boolean formula $F(x)$ in NNF and outputs an equi-satisfiable formula $G(y,x)$ in CNF. Both input and output must be text files 

In [42]:
# NNF = "c this is a comment \\n p cnf 5 31 \\n -5 4 0 \\n -1 5 3 4 0 \\n -3 -4 0" #@param {type:"string"}
class NNFFileDoesNotExist(Exception):
    pass
class MisMatchedBrackets(Exception):
    pass
class InvalidNode(Exception):
    pass

In [155]:
class Formula:
    def __init__(self, root=None):
        self.root = root
        self.vars = {}
        
class Node:
    def __init__(self, val:str = '', typ:str = 'V', children:List = []):
        self.val = val
        self.typ = typ
        if typ != 'V' and typ != 'INVALID' and children == []:
            raise InvalidNode('Non-Variable nodes should have atleast one child')
        self.children = children
    
    @property 
    def is_var(self):
        return self.typ == 'V'
    
    def __str__(self):
        st = []
        st += str(self.val)
        if self.typ != 'V':
            st += '['
            for node in self.children:
                st += node.__str__()
                st += ','
            st[-1] = ']'
        return ''.join(st)
        

In [156]:
print(Node('.', '.', [Node('&', '&', [Node(-1)]), Node(-2), Node(-3)]))

.[&[-1],-2,-3]


In [110]:
def question_1(NNF_filename: str, CNF_filename: str) -> None:
    try:  
        f = open(NNF_filename, 'r')
    except:
        raise NNFFileDoesNotExist
    NNF = f.readline()
    detect_mismatched_brackets(NNF)
    NNF = NNF.replace(' ', '').replace('\n', '')
    NNF_no_brackets = remove_brackets(NNF)
    root = construct_tree(NNF_no_brackets)
    print(f'{NNF}: {NNF_no_brackets}')

In [119]:
def detect_mismatched_brackets(NNF: str):
    st = []
    le = len(NNF)
    for i in range(le):
        char = NNF[i]
        if char == '(':
            st.append(i)
        elif char == ')':
            if st == []:
                raise MisMatchedBrackets(f'Extra closing bracket at index {i+1} in file')
            st.pop()
        
    if st != []:
        raise MisMatchedBrackets(f'Unpaired opening bracket at index {st[0]+1} in file')

In [177]:
def remove_brackets(NNF: str):
    li = ['']
    le = len(NNF)
    i = 0
    while i < le:
        if NNF[i] == '(':
            counter = 0
            flag = True
            for j in range(i+1, le):
                flag = False
                if NNF[j] == '(':
                    counter += 1
                if NNF[j] == ')':
                    if counter == 0:
                        sub_list = remove_brackets(NNF[i+1:j])
                        if li[-1] == '':
                            li.pop()
                        if li == []:
                            li.append(sub_list)
                        else:
                            li.append(sub_list)
                        li.append('')
                        i = j
                        break
                    counter -= 1
        else:
            li[-1] += NNF[i]
        i += 1
    if li[-1] == '':
        li.pop()
    return li

In [175]:
def break_clause(clause: str):
    li = []
    for part_clause in clause.split('.'):
        part_list = []
        print(f'pc: {part_clause}, {isinstance(part_clause, str)}')
        for part_part_clause in part_clause:
            if isinstance(part_part_clause, list):
                part_list.append(part_part_clause.split('+'))
            else:
                part_list.append(part_part_clause)
        li.append(part_list)
    return li

In [179]:
def construct_tree(NNF: List[Any]) -> Node:
    parent_of_root = Node('INVALID', 'INVALID', [Node('INVALID', 'INVALID')])
    last_node = None
    for part_clause in NNF:
        if isinstance(part_clause, list):
            new_node = construct_tree(part_clause)
            new_node = new_node.children[0]
        elif isinstance(part_clause, str):
            broken = break_clause(part_clause)
            print(part_clause, broken)
        else:
            print('INVALID CLAUSE:', part_clause)
    print(parent_of_root)
    return parent_of_root
            
    
def tseitin(NNF: str, CNF: str) -> None:
    root = construct_tree(NNF)
    
question_1('./nnf/0.nnf','./cnf/0.cnf')
question_1('./nnf/1.nnf','./cnf/1.cnf')
question_1('./nnf/4.nnf','./cnf/4.cnf')
question_1('./nnf/2.nnf','./cnf/2.cnf')
try:
    question_1('./nnf/3.nnf','./cnf/3.cnf')
except MisMatchedBrackets:
    print('./nnf/3.nnf: As Expected brackets mismatch')
else:
    print('./nnf/3.nnf: Expected MisMatchedBrackets')
try:
    question_1('./nnf/5.nnf','./cnf/5.cnf')
except MisMatchedBrackets:
    print(f'./nnf/5.nnf: As Expected brackets mismatch')
else:
    print('./nnf/5.nnf: Expected MisMatchedBrackets')

pc: 1+, True
1+ [['1', '+']]
pc: -2, True
pc: , True
-2. [['-', '2'], []]
pc: 3+-4, True
3+-4 [['3', '+', '-', '4']]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
1+(-2.(3+-4)): ['1+', ['-2.', ['3+-4']]]
pc: 1, True
pc: , True
1. [['1'], []]
pc: 2+, True
2+ [['2', '+']]
pc: 3, True
pc: 5, True
3.5 [['3'], ['5']]
INVALID[INVALID]]
pc: +-1, True
pc: 3, True
+-1.3 [['+', '-', '1'], ['3']]
INVALID[INVALID]]
pc: +4, True
pc: 6+12+15, True
+4.6+12+15 [['+', '4'], ['6', '+', '1', '2', '+', '1', '5']]
INVALID[INVALID]]
1.(2+(3.5)+-1.3)+4.6+12+15: ['1.', ['2+', ['3.5'], '+-1.3'], '+4.6+12+15']
pc: -1, True
-1 [['-', '1']]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INVALID]]
INVALID[INV

## Question 2: DPLL SAT Solver (maximum 4 points)
Write a DPLL SAT solver as specified in lecture 2 slides.  Your solver must take as input CNF formulasin  DIMACS  format,  and  determine  their  satisfiability.   Output  has  to  be  the  string  ”SAT”  (respectively, ”UNSAT”) printed to screen, if the input is satisfiable (respectively, unsatisfiable). Perhaps the hardest taskhere is to write the BCP routine, given there are many different kinds of BCP implementations.  Please usethe greedy BCP specified in class.

## Question 3: Solvers and Symbolic Execution-based Testing (maximum 2 points)
Dynamic systematic testing (aka dynamic symbolic testing or concolic testing) is aimed at both 1) automaticsystematic test coverage, and 2) finding deep security vulnerabilities.  For convenience, I will interchange-ably use the term concolic testing and symbolic-execution based testing (or techniques). Symbolic executionis one of the biggest industrial application of SAT solvers.We learnt in class the power of the combination of symbolic execution, concretization and constraintsolvers (e.g., SAT solvers) in automatically finding security vulnerabilities and systematically testing soft-ware.  In this question, we will analyze the role of combining symbolic execution with concrete execution(aka concolic testing), and see how it is more powerful than either symbolic or concrete execution by them-selves. (The term concolic is a portmanteau of concrete and symbolic.)
### Definitions
* Concrete Execution: The term concrete execution refers to the normal execution or run of a program P on a computer on concrete inputs, i.e., the inputs to the program P are values from P’s input domain.
* Program Path or Trace: A program path (simply path or trace) is the sequence of instructions executedby a program on a concrete or symbolic execution